In [1]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mlflow
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
import joblib
import os
from dotenv import dotenv_values

In [45]:
config = dotenv_values("/Users/urkarsh.kulshrestha/Documents/AI_environment/work_env/appletree_end_to_end_forecasting/.env")
print(mlflow.__version__)
import sys
print(sys.executable)

python-dotenv could not parse statement starting at line 1


3.8.1
/Users/urkarsh.kulshrestha/Documents/AI_environment/work_env/appletree_end_to_end_forecasting/appletree_mlops_env/bin/python


In [46]:
'Read credentials to download data'

bucket_name = config.get("bucket_name")
org_name = config.get("org_name")
url = config.get("soracom_url")
influx_token = config.get("influx_token")
print(bucket_name, org_name, url)
client = influxdb_client.InfluxDBClient(url=url, token=influx_token, org=org_name)
query_api = client.query_api()

PXC PXC http://13-192-42-85.napter.soracom.io:27747


In [47]:
fields = [
    "actualNetLoad"
]

fields_filter = " or ".join([f'r["_field"] == "{field}"' for field in fields])

query = f'''
from(bucket: "{bucket_name}")
  |> range(start:0, stop: 2026-01-17T00:00:00Z)
  |> filter(fn: (r) => 
      r["_measurement"] == "FncPvSc"
      )
  |> filter(fn: (r) => {fields_filter})
  |> aggregateWindow(every: 60m, fn: mean, createEmpty: false)
  |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
'''




tables = query_api.query_data_frame(query=query, org=org_name)
#tables['_time'] = pd.to_datetime(tables['_time'], utc=True)
#tables.rename(columns={'_time': 'time'}, inplace=True)


In [6]:
print(tables.head(5))
print(tables.tail(5))

    result  table                    _start                     _stop  \
0  _result      0 1970-01-01 00:00:00+00:00 2026-01-20 00:00:00+00:00   
1  _result      0 1970-01-01 00:00:00+00:00 2026-01-20 00:00:00+00:00   
2  _result      0 1970-01-01 00:00:00+00:00 2026-01-20 00:00:00+00:00   
3  _result      0 1970-01-01 00:00:00+00:00 2026-01-20 00:00:00+00:00   
4  _result      0 1970-01-01 00:00:00+00:00 2026-01-20 00:00:00+00:00   

                       time _measurement  actualNetLoad  
0 2025-03-14 12:00:00+00:00      FncPvSc      31.005559  
1 2025-03-14 13:00:00+00:00      FncPvSc      22.756666  
2 2025-03-14 14:00:00+00:00      FncPvSc       8.218262  
3 2025-03-14 15:00:00+00:00      FncPvSc       6.962737  
4 2025-03-14 16:00:00+00:00      FncPvSc       6.938568  
       result  table                    _start                     _stop  \
7428  _result      0 1970-01-01 00:00:00+00:00 2026-01-20 00:00:00+00:00   
7429  _result      0 1970-01-01 00:00:00+00:00 2026-01-20 00:

In [34]:
temperature_data = pd.read_csv("/Users/urkarsh.kulshrestha/Documents/AI_environment/work_env/appletree_end_to_end_forecasting/data/hourly_temperature_data.csv")
temperature_data.rename(columns={"date": "time"}, inplace=True)
temperature_data['time'] = pd.to_datetime(temperature_data['time'], utc=True)
print(temperature_data.head(5))

                       time  temperature_2m
0 2025-03-14 00:00:00+00:00           10.60
1 2025-03-14 01:00:00+00:00           13.50
2 2025-03-14 02:00:00+00:00           14.75
3 2025-03-14 03:00:00+00:00           15.75
4 2025-03-14 04:00:00+00:00           16.70


In [35]:
'''Merge temperature data with net load data'''

dataset = pd.merge(tables, temperature_data, on="time", how='inner')


In [36]:
dataset

,result,table,_start,_stop,time,_measurement,actualNetLoad,temperature_2m
0,_result,0,1970-01-01 00:00:00+00:00,2026-01-16 00:00:00+00:00,2025-03-14 12:00:00+00:00,FncPvSc,31.005559,7.00
1,_result,0,1970-01-01 00:00:00+00:00,2026-01-16 00:00:00+00:00,2025-03-14 13:00:00+00:00,FncPvSc,22.756666,7.25
2,_result,0,1970-01-01 00:00:00+00:00,2026-01-16 00:00:00+00:00,2025-03-14 14:00:00+00:00,FncPvSc,8.218262,6.60
3,_result,0,1970-01-01 00:00:00+00:00,2026-01-16 00:00:00+00:00,2025-03-14 15:00:00+00:00,FncPvSc,6.962737,6.20
4,_result,0,1970-01-01 00:00:00+00:00,2026-01-16 00:00:00+00:00,2025-03-14 16:00:00+00:00,FncPvSc,6.938568,5.85
...,...,...,...,...,...,...,...,...
7332,_result,0,1970-01-01 00:00:00+00:00,2026-01-16 00:00:00+00:00,2026-01-15 20:00:00+00:00,FncPvSc,6.398484,-0.15
7333,_result,0,1970-01-01 00:00:00+00:00,2026-01-16 00:00:00+00:00,2026-01-15 21:00:00+00:00,FncPvSc,6.583411,-0.05
7334,_result,0,1970-01-01 00:00:00+00:00,2026-01-16 00:00:00+00:00,2026-01-15 22:00:00+00:00,FncPvSc,7.272578,-0.35
7335,_result,0,1970-01-01 00:00:00+00:00,2026-01-16 00:00:00+00:00,2026-01-15 23:00:00+00:00,FncPvSc,7.563843,1.50


In [37]:
dataset = dataset.drop(columns=['_start', '_stop', 'table', 'result'])

In [38]:
dataset

,time,_measurement,actualNetLoad,temperature_2m
0,2025-03-14 12:00:00+00:00,FncPvSc,31.005559,7.00
1,2025-03-14 13:00:00+00:00,FncPvSc,22.756666,7.25
2,2025-03-14 14:00:00+00:00,FncPvSc,8.218262,6.60
3,2025-03-14 15:00:00+00:00,FncPvSc,6.962737,6.20
4,2025-03-14 16:00:00+00:00,FncPvSc,6.938568,5.85
...,...,...,...,...
7332,2026-01-15 20:00:00+00:00,FncPvSc,6.398484,-0.15
7333,2026-01-15 21:00:00+00:00,FncPvSc,6.583411,-0.05
7334,2026-01-15 22:00:00+00:00,FncPvSc,7.272578,-0.35
7335,2026-01-15 23:00:00+00:00,FncPvSc,7.563843,1.50


In [48]:
tables.to_csv("/Users/urkarsh.kulshrestha/Documents/AI_environment/work_env/appletree_end_to_end_forecasting/data/raw/data_without_temperature/mito_dataset_14march_2025_20jan_2026.csv", index=False)